<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/twelvelabs-io/twelvelabs-developer-experience/blob/main/quickstarts/1.0.0-beta/TwelveLabs_Quickstart_Embeddings.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in  Colab</a>
  </td>
</table>

# Create embeddings

This guide shows how to utilize the TwelveLabs Python SDK to create embeddings for your videos.

## Key concepts

- **Asset**: Your uploaded content
- **Embedding**: Vector representation of your content.
- **Embedding task**: An asynchronous operation for processing your content and creating embeddings. Contains a status and the resulting embeddings when complete.

## How it works

To create video embeddings, provide your video content to the platform. You can upload video files as assets, provide a publicly accessible URL, or use base64-encoded data.

The upload method in this guide supports video files up to 4 hours. For details about the available upload methods and the corresponding limits, see the [Upload methods](https://docs.twelvelabs.io/docs/concepts/upload-methods) page.

**Customize your embeddings**

You can customize your embeddings in the following ways:
- Specify the types of embeddings you wish to generate:
    - **Visual**: Based on visual content
    - **Audio**: Based on audio content, excluding spoken words
    - **Transcription**: Based on spoken words extracted from the audio track
- Choose the embedding scope: clip (per segment) or asset (entire video)
- Define how the platform divides your video into segments: dynamic (scene-based) or fixed (time-based)

# Prerequisites

- To use the platform, you need an API key:
  1. If you don't have an account, [sign up](https://playground.twelvelabs.io/) for a free account. No credit card is required to use the Free plan. This plan allows you to index up to 600 minutes of videos, which is sufficient for a small project.
  2. Go to the [API Keys](https://playground.twelvelabs.io/dashboard/api-keys) page.
  3. If you need to create a new key, select the **Create API Key** button. Enter a name and set the expiration period. The default is 12 months.
  4. Select the **Copy** icon next to your key to copy it to your clipboard.
- Your files must meet the [format requirements](https://docs.twelvelabs.io/docs/concepts/models/marengo#input-requirements).


# Procedure

## Install the TwelveLabs Python SDK

In [ ]:
%pip install twelvelabs

## Import the required packages

In [11]:
import time
from twelvelabs import (
    TwelveLabs,
    VideoInputRequest,
    MediaSource,
    # For dynamic segmentation uncomment the next two lines:
    # VideoSegmentation_Dynamic,
    # VideoSegmentationDynamicDynamic,
    # For fixed segmentation uncomment the next two lines:
    # VideoSegmentation_Fixed,
    # VideoSegmentationFixedFixed,
)

## Configure your API key


In [ ]:
# For Google Colab, store your API key as a Secret named `TL_API_KEY`. If you don't know how to create a Colab Secret, see https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75.

from google.colab import userdata
TL_API_KEY = userdata.get('TL_API_KEY')

# For other Python environments, you can use environment variables
# TL_API_KEY = os.environ.get('TL_API_KEY')

## Upload a video

In [ ]:
client = TwelveLabs(api_key=TL_API_KEY)

asset = client.assets.create(
    method="url",
    url="<YOUR_VIDEO_URL>" # Example: https://github.com/twelvelabs-io/twelvelabs-developer-experience/raw/refs/heads/main/quickstarts/steve_jobs_introduces_iphone_in_2007.mp4
    # Or use method="direct" and file=open("<PATH_TO_VIDEO_FILE>", "rb") to upload a file from the local file system
)
print(f"Created asset: id={asset.id}")

Created asset: id=695e214e76cbe75a28db2bd5


## Process your video

In [ ]:
task = client.embed.v_2.tasks.create(
    input_type="video",
    model_name="marengo3.0",
    video=VideoInputRequest(
        media_source=MediaSource(
            asset_id=asset.id,
            # url="<YOUR_VIDEO_URL>",
            # base_64_string="<BASE_64_ENCODED_DATA>",
        ),
        # start_sec=0,
        # end_sec=10,
        # embedding_option=["visual", "audio", "transcription"],
        # embedding_scope=["clip","asset"]
        # For dynamic segmentation:
        # segmentation=VideoSegmentation_Dynamic(
        #     dynamic=VideoSegmentationDynamicDynamic(
        #         min_duration_sec=3  # Minimum segment duration in seconds
        #     )
        # ),
        # For fixed segmentation:
        # segmentation=VideoSegmentation_Fixed(
        #     fixed=VideoSegmentationFixedFixed(
        #         duration_sec=5  # Exact segment duration in seconds
        #     )
        # ),
    ),
)
print(f"Task ID: {task.id}")

## Monitor the status

In [ ]:
while True:
    task = client.embed.v_2.tasks.retrieve(task_id=task.id)
    if task.status == "ready":
        print(f"Task completed")
        break
    elif task.status == "failed":
        print("Task failed")
        break
    else:
        print("Task still processing...")
        time.sleep(5)


## Process the results

In [ ]:
print(f"\n{'='*80}")
print(f"EMBEDDINGS SUMMARY: {len(task.data)} total embeddings")
print(f"{'='*80}\n")
for idx, embedding_data in enumerate(task.data, 1):
    print(f"[{idx}/{len(task.data)}] {embedding_data.embedding_option.upper()} | {embedding_data.embedding_scope.upper()}")
    print(f"├─ Time range: {embedding_data.start_sec}s - {embedding_data.end_sec}s")
    print(f"├─ Dimensions: {len(embedding_data.embedding)}")
    print(f"└─ First 10 values: {embedding_data.embedding[:10]}")
    print()

# Next steps

For a comprehensive guide on creating video and other types of embeddings, see the [Create embeddings](https://docs.twelvelabs.io/docs/guides/create-embeddings) page.